In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [2]:
cd ..\experiments\data\human\1.7

C:\Users\piichan\Documents\Berkeley\CoCoSci Lab\mcrl\mcrl\experiments\data\human\1.7


In [3]:
df=pd.read_csv('trials.csv', sep=',')

In [4]:
import ast
toList = lambda x: ast.literal_eval(x)
df['clicks'] = df['clicks'].apply(toList)

In [5]:
df.info_cost.unique()

array([ 0.1 ,  1.25,  4.  ])

In [6]:
df.sort_values('info_cost')
low_cost = df.loc[df.info_cost == 0.1]
med_cost = df.loc[df.info_cost == 1.25]
high_cost = df.loc[df.info_cost == 4]

In [7]:
count = 0
for seq in df['clicks']:
    if seq == []:
        count += 1
        
count/df['clicks'].size

0.30399568034557234

In [8]:
cost_type = low_cost
brfs_count = 0
dfs_count = 0
top = set([9, 10, 11, 12])
left = set([13, 14, 15, 16])
right = set([5,6,7,8])
bottom = set([1, 2, 3, 4])
for seq in cost_type['clicks']:
    if seq == []:
        continue     
    if len(seq) < 4:
        continue
    first_four = seq[:4]
    if set(first_four) == set([1,5,9,13]):
        if len(seq) > 4:
            rest = seq[4:]
            if len(rest) > 4:
                rest = seq[4:8]
            if set(rest) <= set([2,6,10,14]):
                brfs_count += 1 
        else:
            brfs_count += 1
    elif (set(first_four) == top)or(set(first_four) == right)or(set(first_four) == bottom)or(set(first_four) == left):
        if len(seq) > 4:
            rest = set(seq[4:])
            more = False
            if len(rest) > 4:
                more = True
                rest = set(seq[4:8])
            if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                if more:
                    rest = set(seq[8:])
                    if len(rest) > 4:
                        rest = set(seq[8:12])
                    if (rest<=top)or(rest<=right)or(rest<=left)or(rest<=bottom):
                            dfs_count += 1                    
                else:
                    dfs_count += 1
        else:
            dfs_count += 1
    else: #between 1 and 3 clicks were made
        if (set(seq) < top)or(set(seq) < right)or(set(seq)< bottom) or (set(seq) < left):
            dfs_count += 1
        elif set(seq) < set([1, 5, 9, 13]):
            brfs_count += 1
        
(brfs_count/cost_type['clicks'].size) * 100

5.0245098039215685

ver 0.991 

values are given in percentages; top to bottom is low, med, and high cost

i stands for initial (only the first 4 clicks) while f stands for full (entire sequence)

| Breadth-First | Depth-First(i)| Depth-First(f)|
| ------------- |:-------------:| -------------:|
| 3.02          | 12.50         |8.47       |
| 6.07          | 12.13         |    9.375      |
| 4.23          | 3.43          |    2.42       |



In [1]:
def depth_first(state, last_click, dist):
    prob_next_click = np.zeros(17)
    observed_states = np.nonzero(state)[0]
    possible_clicks = []
    if last_click == None:
        possible_clicks += [1, 5, 9, 13]
    elif last_click in [1, 5, 9, 13]:
        if last_click+1 in observed_states:
            for click in [1, 5, 9, 13]:
                if click not in observed_states:
                    possible_clicks.append(click)                 
        else:
            possible_clicks.append(last_click+1)
    else:
        if last_click+1 in observed_states:
            if last_click+2 in observed_states:
                for click in [1, 5, 9, 13]:
                    if click not in observed_states:
                        possible_clicks.append(click)
            elif (last_click+2 < 17):
                possible_clicks.append(last_click+2)
            else:
                for click in [1, 5, 9, 13]:
                    if click not in observed_states:
                        possible_clicks.append(click)
        elif (last_click+1 < 17):
            possible_clicks.append(last_click+1)
        else:
            for click in [1, 5, 9, 13]:
                    if click not in observed_states:
                        possible_clicks.append(click)
    if len(possible_clicks) != 0:
        for possibility in possible_clicks:
            if dist == 'Uniform':
                prob_next_click[possibility] = 1/len(possible_clicks)
    return prob_next_click

In [4]:
test_state = [0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1]
res = depth_first(test_state, 16, 'Uniform')
res

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ,
        0. ,  0. ,  0.5,  0. ,  0. ,  0. ])

In [38]:
def strategy_likelihood(click_seq, strategy, p_error, log=False, dist='Uniform', stop=None):
    prob = 0
    cur_state = np.zeros(17)
    last_click = None
    for click in click_seq:
        possible_next_clicks = strategy(cur_state, last_click, dist)
        next_state = np.copy(cur_state)
        next_state[click] = 1
        last_click = click
        cur_state = next_state
        n_possible_clicks = np.count_nonzero(possible_next_clicks)
        for p_click in possible_next_clicks:
            if p_click == click:
                p_strat = p_click
            else:
                p_strat = 0
        num_wrong_clicks = 16 - (np.count_nonzero(cur_state)+n_possible_clicks)
        if dist == 'Uniform':
            p_click_given_state = (((1-p_error)*p_strat) + (p_error/num_wrong_clicks))
        if log:
            prob += np.log(p_click_given_state)
        else:
            prob *= p_click_given_state
    return prob